# Fabric APIs
This notebook is an example of how to use a lot of the APIs in OneLake (ADLSGen2) and Fabric and Azure Management. I will try to keep these up-to-date with others. I just like having an example of everything in one place 😀

[OneLake](#onelake-apis)

[Upload a File](#upload-a-file)

[Load File to Table](#load-file-to-table-with-lakehouse-api)

[Query Table](#call-sql-endpoint)



In [ ]:
from azure.identity import DefaultAzureCredential
import requests
import json,os
import struct
from itertools import chain, repeat
import pyodbc

## Get Credentials for Future Calls

In [ ]:
credential = DefaultAzureCredential() 

# Fabric token
fabric_token=credential.get_token('https://api.fabric.microsoft.com/.default')
fabric_headers={'Authorization': 'Bearer ' + fabric_token.token, 'Content-Type': 'application/json'}

# Storage token
onelake_token=credential.get_token('https://storage.azure.com/.default')
onelake_headers={'Authorization': 'Bearer ' + onelake_token.token, 'Content-Type': 'application/json'}

database_token = credential.get_token("https://database.windows.net/.default") 

In [ ]:
# CHANGE THESE TO YOUR OWN: Setup the variables for the workspace and lakehouse
workspaceName="fabricit"
lakehouseName="mslearn"

## Get the Workspace ID

In [ ]:
# Use the old Power BI API to get the workspace ID
# NOTE: This is on the roadmap to provide search 
# - https://learn.microsoft.com/en-us/fabric/release-plan/shared-experiences#workspace-filters-search-support-nested-folders
response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups?$filter=tolower(name) eq tolower('{workspaceName}')", headers=fabric_headers)
# response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups?$filter=startswith(name,'{workspaceName}')", headers=fabric_headers)
# response = requests.get("https://api.powerbi.com/v1.0/myorg/groups", headers=fabric_headers)

if 200 == response.status_code:
    values=response.json()['value']
    if len(values)==1:
        workspaceId=values[0]['id']
        print("workspaceId",workspaceId)
    else:
        print("Workspace not found")


## Get the Lakehouse ID

In [ ]:
# response = requests.get("https://api.fabric.microsoft.com/v1/workspaces?$filter=contains(displayName,'fabricit')", headers=fabric_headers)
response = requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/Lakehouses", headers=fabric_headers)
print(response.status_code)
# Grab the one that matches the name of lakehouseName
if 200 == response.status_code:
    for lakehouse in response.json()['value']:
        # case insensitive comparison
        if lakehouse['displayName'].lower() == lakehouseName.lower():
            lakehouseId = lakehouse['id']
            print("lakehouseId",lakehouseId)
            break


# OneLake APIs
You can access by either the names or the GUIDs. HOWEVER, you cannot have spaces in the workspace (aka Container) or it won't work at all by name

> NOTE: If you access by names, you can't interchange GUID and Names

References:
- https://learn.microsoft.com/en-us/fabric/onelake/onelake-access-python
- https://learn.microsoft.com/en-us/fabric/onelake/onelake-access-api

## Get Items by Name

In [ ]:
# Get items by the name of the workspace
requests.get(f"https://onelake.dfs.fabric.microsoft.com/{workspaceName}?resource=filesystem&recursive=false", headers=onelake_headers).json()

## Get Items by GUID
> NOTE: See the changes in the response are GUIDs and not names

In [ ]:
# Get items by the name of the workspace
requests.get(f"https://onelake.dfs.fabric.microsoft.com/{workspaceId}?resource=filesystem&recursive=false", headers=onelake_headers).json()

## Upload a File

In [ ]:
# Upload the demo.csv file to OneLake
url = f"https://onelake.dfs.fabric.microsoft.com/{workspaceId}/{lakehouseId}/Files/demo.csv"

headers=onelake_headers.copy()
headers['Content-Length']="0"

with open('demo.csv', 'rb') as f:
    response = requests.put(url, headers=headers, params={'resource': 'file'})

# Check the response
if response.status_code == 201:
    print("File creation successful")
else:
    print("File creation failed:", response.text)

# Upload the data
headers=onelake_headers.copy()
headers['Content-Length'] = str(os.path.getsize('demo.csv'))  # Set the Content-Length to the size of the file
with open('demo.csv', 'rb') as f:
    response = requests.patch(url, headers=headers, data=f, params={'action': 'append', 'position': '0'})

if response.status_code == 202:
    print("Data upload successful")
else:
    print("Data upload failed:", response.text)

# Flush the data
headers=onelake_headers.copy()
headers['Content-Length'] = "0"
response = requests.patch(url, headers=headers, params={'action': 'flush', 'position': str(os.path.getsize('demo.csv'))})

if response.status_code == 200:
    print("Data flush successful")
else:
    print("Data flush failed:", response.text)    

In [ ]:
# Get items by the name of the workspace
requests.get(f"https://onelake.dfs.fabric.microsoft.com/{workspaceName}/{lakehouseName}.Lakehouse/Files?resource=filesystem&recursive=false", headers=onelake_headers).json()

# Get the file properties only
resp=requests.head(f"https://onelake.dfs.fabric.microsoft.com/{workspaceName}/{lakehouseName}.Lakehouse/Files/demo.csv?resource=file", headers=onelake_headers)
print(resp.status_code)
print(resp.headers)

## Download the file

In [ ]:
# Get the actual file
resp=requests.get(f"https://onelake.dfs.fabric.microsoft.com/{workspaceName}/{lakehouseName}.Lakehouse/Files/demo.csv", headers=onelake_headers)
print(resp.status_code)
# Check if the request was successful
if resp.status_code == 200:
    # Write the content to a file
    with open('demo_download.csv', 'wb') as f:
        f.write(resp.content)
    print("File downloaded successfully")
else:
    print("File download failed:", resp.status_code)

# Load File to Table with Lakehouse API

In [ ]:
# Load Table from File
body = { 
  "relativePath": "Files/demo.csv", 
  "pathType": "File", 
  "mode": "overwrite", 
  "recursive": False,
  "formatOptions": 
  { 
    "header": True, 
    "delimiter": ",", 
    "format": "CSV" 
  } 
}
response = requests.post(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses/{lakehouseId}/tables/demo/load", headers=fabric_headers, json=body)
print(response.status_code)

if 202 == response.status_code:
    print("Table load started")
    response.headers["Location"]

In [ ]:
# Check the status of the load
url = response.headers["Location"]
response = requests.get(url, headers=fabric_headers)
print(response.status_code)
print(response.json())

In [ ]:
# Get items by the name of the workspace
requests.get(f"https://onelake.dfs.fabric.microsoft.com/{workspaceName}/{lakehouseName}.Lakehouse/Files?resource=filesystem&recursive=false", headers=onelake_headers).json()

# Fabric REST API

In [ ]:
# Review the properties of a Workspace
requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}", headers=fabric_headers).json()

In [ ]:
# Get the Lakehouses in a Workspace
requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/Lakehouses", headers=fabric_headers).json()

# Call SQL Endpoint

In [ ]:
# Get the lakehouse
resp = requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/Lakehouses/{lakehouseId}", headers=fabric_headers).json()
# print(json.dumps(resp, indent=2))

lakehouseEndpoint = resp['properties']['sqlEndpointProperties']['connectionString']
databaseName=resp['displayName']

# Load of credentials and execute the code

credential = DefaultAzureCredential() 
sql_endpoint = lakehouseEndpoint 
database = databaseName 

connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Database=f{database};Encrypt=Yes;TrustServerCertificate=No"

# Convert the token to a UTF-8 byte string
token_as_bytes = bytes(database_token.token, "UTF-8") 
# Encode the bytes to a Windows byte string
encoded_bytes = bytes(chain.from_iterable(zip(token_as_bytes, repeat(0)))) 
# Package the token into a bytes object
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes 
# Attribute pointing to SQL_COPT_SS_ACCESS_TOKEN to pass access token to the driver
attrs_before = {1256: token_bytes}  

connection = pyodbc.connect(connection_string, attrs_before=attrs_before)
cursor = connection.cursor()
# cursor.execute("SELECT * FROM sys.tables")
cursor.execute(f"SELECT * FROM {lakehouseName}.dbo.demo")
rows = cursor.fetchall()
print(rows)

cursor.close()
connection.close()

# Data Access Roles
https://learn.microsoft.com/en-us/fabric/onelake/security/get-started-data-access-roles
- You must opt in on the Lakehouse for these permissions

https://blog.fabric.microsoft.com/en-us/blog/onelake-data-access-roles-apis-announcement

https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-data-access-security/list-data-access-roles?tabs=HTTP


In [ ]:
requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{lakehouseId}/dataAccessRoles", headers=fabric_headers).json()
